In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
'''
Dropout Regularization : randomly drop out certain neurons in a specific layer by 
a specified rate. By doing so we are removing the  bias of one neuron towards the other.
'''

'\nDropout Regularization : randomly drop out certain neurons in a specific layer by \na specified rate. By doing so we are removing the  bias of one neuron towards the other.\n'

# IMPORTING DATA

In [4]:
import requests
url = 'https://raw.githubusercontent.com/codebasics/deep-learning-keras-tf-tutorial/master/13_dropout_layer/sonar_dataset.csv'
res = requests.get(url, allow_redirects=True)
with open('sonar_dataset.csv','wb') as file:
    file.write(res.content)
df = pd.read_csv('sonar_dataset.csv', header=None)

df.sample(6)

,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,60
118,0.0363,0.0478,0.0298,0.0210,0.1409,0.1916,0.1349,0.1613,0.1703,0.1444,...,0.0115,0.0190,0.0055,0.0096,0.0050,0.0066,0.0114,0.0073,0.0033,M
195,0.0129,0.0141,0.0309,0.0375,0.0767,0.0787,0.0662,0.1108,0.1777,0.2245,...,0.0124,0.0093,0.0072,0.0019,0.0027,0.0054,0.0017,0.0024,0.0029,M
51,0.0131,0.0068,0.0308,0.0311,0.0085,0.0767,0.0771,0.0640,0.0726,0.0901,...,0.0062,0.0028,0.0040,0.0075,0.0039,0.0053,0.0013,0.0052,0.0023,R
7,0.0519,0.0548,0.0842,0.0319,0.1158,0.0922,0.1027,0.0613,0.1465,0.2838,...,0.0081,0.0120,0.0045,0.0121,0.0097,0.0085,0.0047,0.0048,0.0053,R
148,0.0712,0.0901,0.1276,0.1497,0.1284,0.1165,0.1285,0.1684,0.1830,0.2127,...,0.0154,0.0156,0.0054,0.0030,0.0048,0.0087,0.0101,0.0095,0.0068,M
190,0.0156,0.0210,0.0282,0.0596,0.0462,0.0779,0.1365,0.0780,0.1038,0.1567,...,0.0150,0.0060,0.0082,0.0091,0.0038,0.0056,0.0056,0.0048,0.0024,M


# Data Manuplation

In [6]:
df.shape

(208, 61)

In [17]:
df.columns

Int64Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
            17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
            34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
            51, 52, 53, 54, 55, 56, 57, 58, 59, 60],
           dtype='int64')

In [18]:
df.isna().sum()

0     0
1     0
2     0
3     0
4     0
     ..
56    0
57    0
58    0
59    0
60    0
Length: 61, dtype: int64

# Training ANN

In [20]:
#Splitting data into input and output
x = df.drop(60,axis='columns')
y = df[60]
y.head()

0    R
1    R
2    R
3    R
4    R
Name: 60, dtype: object

In [29]:
#Converting Y into 1/0 format:
y2 = df[60]
replaced_words = {
    'R' : 1,
    'M' : 0
}
y2 = y2.map(replaced_words)

In [30]:
y2

0      1
1      1
2      1
3      1
4      1
      ..
203    0
204    0
205    0
206    0
207    0
Name: 60, Length: 208, dtype: int64

In [32]:
#Dividing Data into train test split:
from sklearn.model_selection import train_test_split
x_train , x_test , y_train , y_test = train_test_split(x , y2 , test_size = 0.25 , random_state = 5)

In [33]:
x_train.sample(5)

,0,1,2,3,4,5,6,7,8,9,...,50,51,52,53,54,55,56,57,58,59
23,0.0115,0.0150,0.0136,0.0076,0.0211,0.1058,0.1023,0.0440,0.0931,0.0734,...,0.0107,0.0091,0.0016,0.0084,0.0064,0.0026,0.0029,0.0037,0.0070,0.0041
50,0.0353,0.0713,0.0326,0.0272,0.0370,0.0792,0.1083,0.0687,0.0298,0.0880,...,0.0098,0.0163,0.0242,0.0043,0.0202,0.0108,0.0037,0.0096,0.0093,0.0053
163,0.0072,0.0027,0.0089,0.0061,0.0420,0.0865,0.1182,0.0999,0.1976,0.2318,...,0.0092,0.0078,0.0071,0.0081,0.0034,0.0064,0.0037,0.0036,0.0012,0.0037
78,0.0231,0.0351,0.0030,0.0304,0.0339,0.0860,0.1738,0.1351,0.1063,0.0347,...,0.0154,0.0106,0.0097,0.0022,0.0052,0.0072,0.0056,0.0038,0.0043,0.0030
60,0.0130,0.0006,0.0088,0.0456,0.0525,0.0778,0.0931,0.0941,0.1711,0.1483,...,0.0092,0.0078,0.0041,0.0013,0.0011,0.0045,0.0039,0.0022,0.0023,0.0016


In [34]:
#Building ANN:
import keras
from keras import Sequential
from keras.layers import Dense

In [35]:
x_train.shape

(156, 60)

In [37]:
model = Sequential([
    Dense(60 , input_dim = 60 , activation='relu'),
    Dense(30 , activation='relu'),
    Dense(15 , activation='relu'),
    Dense(1 , activation='sigmoid'),
])

In [38]:
model.compile(
    optimizer = 'adam',
    loss = 'binary_crossentropy',
    metrics = ['accuracy']
)

In [40]:
model.fit(x_train , y_train , epochs=50, batch_size = 8)
#Batch size is for mini gradient descent
#Accuracy 1 means model has overfit the data!!!!!!

Epoch 1/50
20/20 [==============================] - 0s 2ms/step - loss: 0.2314 - accuracy: 0.9038
Epoch 2/50
20/20 [==============================] - 0s 2ms/step - loss: 0.2249 - accuracy: 0.9167
Epoch 3/50
20/20 [==============================] - 0s 2ms/step - loss: 0.1915 - accuracy: 0.9167
Epoch 4/50
20/20 [==============================] - 0s 2ms/step - loss: 0.1403 - accuracy: 0.9679
Epoch 5/50
20/20 [==============================] - 0s 2ms/step - loss: 0.1315 - accuracy: 0.9551
Epoch 6/50
20/20 [==============================] - 0s 2ms/step - loss: 0.1386 - accuracy: 0.9615
Epoch 7/50
20/20 [==============================] - 0s 2ms/step - loss: 0.1063 - accuracy: 0.9679
Epoch 8/50
20/20 [==============================] - 0s 2ms/step - loss: 0.0973 - accuracy: 0.9808
Epoch 9/50
20/20 [==============================] - 0s 2ms/step - loss: 0.0944 - accuracy: 0.9744
Epoch 10/50
20/20 [==============================] - 0s 2ms/step - loss: 0.0993 - accuracy: 0.9744
Epoch 11/50
20/20 [

In [41]:
model.evaluate(x_test,y_test)

2/2 [==============================] - 0s 4ms/step - loss: 0.4528 - accuracy: 0.8654


[0.4527840316295624, 0.8653846383094788]

In [46]:
y_pred = model.predict(x_test).reshape(-1)
print(y_pred[:10])

#Rounding off to nearest int i.e, 0 or 1 :
y_pred2 = np.round(y_pred)
print(y_pred2[:10])

2/2 [==============================] - 0s 3ms/step
[2.0615418e-05 9.9881959e-01 1.7571788e-02 4.8899200e-02 9.9650908e-01
 6.3615153e-04 7.4721390e-01 9.9999863e-01 8.4642984e-04 9.9998271e-01]
[0. 1. 0. 0. 1. 0. 1. 1. 0. 1.]


In [47]:
#Now we Try to add dropout layer:
from keras.layers import Dropout

In [56]:
model1 = Sequential([
    Dense(60 , input_dim = 60 , activation='relu'),
    Dropout(0.2),
    Dense(30 , activation='relu'),
    Dropout(0.15),
    Dense(15 , activation='relu'),
    Dropout(0.1),
    Dense(1 , activation='sigmoid'),
])

model1.compile(
    optimizer = 'adam',
    loss = 'binary_crossentropy',
    metrics = ['accuracy']
)

model1.fit(x_train,y_train,epochs=100)

Epoch 1/100
5/5 [==============================] - 1s 2ms/step - loss: 0.7099 - accuracy: 0.4872
Epoch 2/100
5/5 [==============================] - 0s 3ms/step - loss: 0.6986 - accuracy: 0.5449
Epoch 3/100
5/5 [==============================] - 0s 3ms/step - loss: 0.6761 - accuracy: 0.5769
Epoch 4/100
5/5 [==============================] - 0s 3ms/step - loss: 0.6684 - accuracy: 0.5962
Epoch 5/100
5/5 [==============================] - 0s 4ms/step - loss: 0.6694 - accuracy: 0.5769
Epoch 6/100
5/5 [==============================] - 0s 3ms/step - loss: 0.6561 - accuracy: 0.6218
Epoch 7/100
5/5 [==============================] - 0s 3ms/step - loss: 0.6476 - accuracy: 0.6282
Epoch 8/100
5/5 [==============================] - 0s 2ms/step - loss: 0.6514 - accuracy: 0.6282
Epoch 9/100
5/5 [==============================] - 0s 2ms/step - loss: 0.6476 - accuracy: 0.6538
Epoch 10/100
5/5 [==============================] - 0s 3ms/step - loss: 0.6300 - accuracy: 0.6923
Epoch 11/100
5/5 [===========

5/5 [==============================] - 0s 2ms/step - loss: 0.1993 - accuracy: 0.9231
Epoch 85/100
5/5 [==============================] - 0s 2ms/step - loss: 0.1804 - accuracy: 0.9295
Epoch 86/100
5/5 [==============================] - 0s 2ms/step - loss: 0.1721 - accuracy: 0.9359
Epoch 87/100
5/5 [==============================] - 0s 2ms/step - loss: 0.1531 - accuracy: 0.9487
Epoch 88/100
5/5 [==============================] - 0s 2ms/step - loss: 0.1531 - accuracy: 0.9551
Epoch 89/100
5/5 [==============================] - 0s 2ms/step - loss: 0.1679 - accuracy: 0.9295
Epoch 90/100
5/5 [==============================] - 0s 2ms/step - loss: 0.1988 - accuracy: 0.8910
Epoch 91/100
5/5 [==============================] - 0s 2ms/step - loss: 0.1562 - accuracy: 0.9359
Epoch 92/100
5/5 [==============================] - 0s 2ms/step - loss: 0.1575 - accuracy: 0.9487
Epoch 93/100
5/5 [==============================] - 0s 2ms/step - loss: 0.1976 - accuracy: 0.9231
Epoch 94/100
5/5 [===============

In [57]:
model1.evaluate(x_test,y_test)

2/2 [==============================] - 0s 3ms/step - loss: 0.3196 - accuracy: 0.8462


[0.31960099935531616, 0.8461538553237915]